In [1]:
import os
import sys
sys.path.append('../../cnn_design-space/cnnbench/')

import numpy as np
import yaml
from six.moves import cPickle as pickle
import hashlib
import subprocess

from library import GraphLib, Graph
from utils import print_util as pu
import manual_models

In [2]:
config_file_path = '../../cnn_design-space/cnnbench/tests/config_all-ops.yaml'

models_mini = ['vgg11', 'vgg13', 'vgg16', 'vgg19', 'resnet18', 'resnet34', 
               'resnet50', 'resnet101', 'resnet152', 'shufflenet', 'mobilenet', 'googlenet',
               'inception', 'xception']

with open(config_file_path) as config_file:
    try:
        config = yaml.safe_load(config_file)
    except yaml.YAMLError as exc:
        print(exc)

ops = []

for model_name in models_mini:
    model = manual_models.get_manual_graph(config, model_name=model_name)
    for module in model.graph:
        ops.extend(module[1])
        
ops = list(set(ops))

for op in ops:
    assert op in config['base_ops'] + config['dense_ops'] + config['flatten_ops'] \
        + ['dense_classes', 'input', 'output'], \
        f'{op} not in config'
    
print('No ops found outside config')

No ops found outside config


In [3]:
dataset_file_path = '../../cnn_design-space/cnnbench/dataset/dataset_mini.json'
config_file_path = '../../cnn_design-space/cnnbench/tests/config_all-ops.yaml'

graphLib = GraphLib.load_from_dataset(dataset_file_path)

with open(config_file_path) as config_file:
    try:
        config = yaml.safe_load(config_file)
    except yaml.YAMLError as exc:
        print(exc)

models_mini = ['vgg11', 'vgg13', 'vgg16', 'vgg19', 'resnet18', 'resnet34', 
               'resnet50', 'resnet101', 'resnet152', 'shufflenet', 'mobilenet', 'googlenet',
               'inception', 'xception']

# Create a dictionary of hashes for all manual models in dataset_mini
manual_models_hashes = {}
graphLib_hashes = [graph.hash for graph in graphLib.library]
for model_name in models_mini:
    model = manual_models.get_manual_graph(config, model_name=model_name)
    manual_models_hashes[model_name] = model.hash
    
    # Check if this model is in graphLib
    assert model.hash in graphLib_hashes, f'{model_name} is not in graphLib'
    
    print(f'{model_name}: {model.hash}')

vgg11: ad0a60f4a4077a823e03c806612b1739d7dbdcb9955a77b582d18ba9eea0a6b8
vgg13: 6d0862109b4bd4ba489ae97bc9f675734fc2a6a54a210b3a4f391fa01889a982
vgg16: c02c2fde320d179d81af17833c9466bfbfbb1facb42d82e14e9793a9b1ae7979
vgg19: 2bc7895ec217f994d1d1bec034626be9a9f68848c97d62f7e1c493cc8c73330b
resnet18: 1da03cb835aa34f8ccbe0ecf6a025ff9e4f8536e1e1713e9a93c65523b7d885a
resnet34: 0b46c0d8566133b428ec736bea72c8b7eda7e79d477f54c587c1f6a521611998
resnet50: 3abff4667b9e17a30f71ad57a6f7d52d217283984e81a878fceabeb43d299114
resnet101: 4021dbdb0a51fcfab78d87515f07ece1227696d97260b019a76810ba60f5ed03
resnet152: e1aff753f15c8284429fe3e6e9a713d6367e151329576d3701967385b17b48e8
shufflenet: 1a135065feaac7541e71be5767980aafb970e67aa6b938a68235192c823bf3ff
mobilenet: 03c30e238db867874e66e985b6fd494d27661cd17f6ceef332415a5af88b647e
googlenet: 7ea48ef10963fb2a794613dfb2e95e5d6c949217fddb17611cb1861951394232
inception: 7ea48ef10963fb2a794613dfb2e95e5d6c949217fddb17611cb1861951394232
xception: 3227a974145e3074af1f

In [4]:
accel_embeddings_file_path = '../../accelerator_design-space/accelbench/embeddings/embeddings.pkl'

# Initialize Accelerator embeddings
accel_embeddings = pickle.load(open(accel_embeddings_file_path, 'rb'))
accel_embeddings = np.array(accel_embeddings)

print(f'Size of the dataset: {accel_embeddings.shape[0]}')

# Check if SPRING is there in accel_embeddings
spring_embedding = np.array([2e0, 16e0, 8e0, 4e0, 8e0, 3e0, 3e0, 1e0, 12e0, 24e0, 4e0, 1e0, 1e0])
spring_present = False

for accel_idx in range(accel_embeddings.shape[0]):
    if (spring_embedding == accel_embeddings[accel_idx, :]).all(): spring_present = True
        
print(f'SPRING present in the dataset: {spring_present}')

Size of the dataset: 57108480
SPRING present in the dataset: False


In [33]:
# Run AccelBench simulations with SPRING and all CNN manual models

models_dir = '/scratch/gpfs/stuli/accelerator_co-design'
cnn_models_dir = os.path.join(models_dir, 'cnnbench_models', config['dataset'])
accel_models_dir = os.path.join(models_dir, 'accelbench_models')

for cnn_model_name, cnn_model_hash in manual_models_hashes.items():
    accel_cnn_str = str(spring_embedding).replace('\n', '') \
        + graphLib.get_graph(model_hash=cnn_model_hash)[0].hash
    accel_hash = hashlib.sha256(accel_cnn_str.encode('utf-8')).hexdigest()
    print(f'accel_hash for SPRING and {cnn_model_name}: {accel_hash}')
    
    args = ['--dataset', config['dataset']]
    args.extend(['--cluster', 'della'])
    args.extend(['--id', 'stuli'])
    args.extend(['--autotune', '0'])
    args.extend(['--cnn_model_hash', cnn_model_hash])
    args.extend(['--cnn_model_dir', os.path.join(cnn_models_dir, cnn_model_hash)])
    args.extend(['--cnn_config_file', config_file_path])
    args.extend(['--graphlib_file', dataset_file_path])
    args.extend(['--train_cnn', '0'])
    args.extend(['--accel_hash', accel_hash])
    args.extend(['--accel_emb', '\\"' + str(spring_embedding).replace('\n', '')[1:-1].replace(',', '') + '\\"'])
    args.extend(['--accel_model_file', os.path.join(accel_models_dir, accel_hash) + '.pkl'])
    
    slurm_stdout = subprocess.check_output(
        f'ssh della-gpu "cd accelerator_co-design/boshcode; source ./job_scripts/job_worker.sh {" ".join(args)}"',
        shell=True, text=True, executable="/bin/bash")

    print(f'Job ID: {slurm_stdout.split()[-1]}\n')

accel_hash for SPRING and vgg11: 08a0734b7c753fd8940a5977dfe968cebfe7f5d0a0d6af13837b7c0623bb4b72
Job ID: 39257903

accel_hash for SPRING and vgg13: fa909baf0301c2d43cb80c4935fea98b4a3a13fd97185428fdda12b1d1d0d080
Job ID: 39257904

accel_hash for SPRING and vgg16: 02bd3dacf4b06d0e078d0453d39e501271f350fc01c25f716f92bb2bd2f7f2b5
Job ID: 39257905

accel_hash for SPRING and vgg19: 19bf3b9956047868d1b5546f69595d16b86f440778b28908ab2a61e16849859c
Job ID: 39257906

accel_hash for SPRING and resnet18: d8538572e9c68de1d836917553c56899474ed7b2a678094a1b80536f384ed60b
Job ID: 39257907

accel_hash for SPRING and resnet34: 339a5e3d9d2e22ef9c88b1f39df246598c457684643fd6ce8cdd1db5df4b83ea
Job ID: 39257908

accel_hash for SPRING and resnet50: 27061773536be6deea1e3e9a4e6eb4f19331b87548de395ac0fa1b7fbef571ee
Job ID: 39257909

accel_hash for SPRING and resnet101: 05757f512c3da8cb7188c7faf010668ac94663099b72cb22de1667ed54294ca2
Job ID: 39257910

accel_hash for SPRING and resnet152: 33e627ae0e10b466a3732c